# Bitcoin Market Prediction uisng CoinmarketCap
An AI-powered project using historical CoinMarketCap data to predict Bitcoin price trends and offer actionable insights for traders.

## Imports

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

## .env configuration

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

### This line creates an authenticated OpenAI client instance, used to make API requests in your code.

In [ ]:
from openai import OpenAI
openai = OpenAI(api_key=api_key)

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
system_prompt = """
You are an intelligent assistant specialized in Bitcoin market prediction. Your tasks are:

- Collect, preprocess, and analyze historical Bitcoin price and volume data sourced from CoinMarketCap historical data tables or API.
- Extract relevant time series and technical features from OHLC (open, high, low, close) and volume data.
- Use machine learning or statistical models to forecast future Bitcoin price trends.
- Output clear, concise, and actionable insights, focusing on predicted price direction and potential trading signals.
- Ensure all data collection respects CoinMarketCap’s terms of service.
- Present findings in user-friendly language, explaining prediction confidence and market risks.
- Continuously improve prediction accuracy through back-testing on updated datasets.

"""


In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are analyzing historical Bitcoin market data from the webpage titled '{website.title}'.\n"
    user_prompt += (
        "The data includes daily open, high, low, close prices, trading volume, "
        "and market capitalization presented in a table format.\n"
        "Please provide a clear and concise analysis in Markdown format, focusing on recent trends, "
        "price movements, volatility, and any insights that could help forecast Bitcoin price directions.\n"
        "If possible, include technical indicators, significant patterns, or notable market events mentioned in the data.\n\n"
    )
    user_prompt += website.text
    return user_prompt


In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

# A function to display this nicely in the Jupyter output, using markdown

def display_summary(summary):    
    display(Markdown(summary))

In [ ]:
url = "https://coinmarketcap.com/currencies/bitcoin/historical-data/3"
summary = summarize(url)
display_summary(summary)